In [1]:
import keras
from tensorflow.keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation, InputLayer,GlobalAveragePooling2D
batch_size_train = 20
datagen=keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())
train_path = '/Users/utkarsh.verma/Downloads/archive/seg_train'
classes_required =['buildings','forest','glacier','mountain','sea','street']
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(150,150), classes=classes_required, batch_size=batch_size_train)
type(train_batches)

Found 14034 images belonging to 6 classes.


tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [2]:

'''Input the Validation Data
'''

val_path = '/Users/utkarsh.verma/Downloads/archive/seg_test'
val_batches = ImageDataGenerator().flow_from_directory(val_path, target_size=(150,150), classes=classes_required, batch_size = 10 )

Found 3000 images belonging to 6 classes.


In [3]:
input_tensor = Input(shape=(150, 150, 3))
vgg16_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet')
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=vgg16_model.input, outputs=predictions)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)  

In [4]:

for layer in vgg16_model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

In [5]:
# Complie the model
from keras.optimizers import Adam
model.compile(Adam(lr=.00015), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit_generator(train_batches, steps_per_epoch=20, 
                    validation_data=val_batches, validation_steps=20, epochs=10, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
20/20 [==============================] - 38s 2s/step - loss: 1.5483 - accuracy: 0.3525 - val_loss: 1.3257 - val_accuracy: 0.4650
Epoch 2/10
20/20 [==============================] - 42s 2s/step - loss: 1.1731 - accuracy: 0.5825 - val_loss: 0.9266 - val_accuracy: 0.6350
Epoch 3/10
20/20 [==============================] - 41s 2s/step - loss: 0.7485 - accuracy: 0.7050 - val_loss: 0.6911 - val_accuracy: 0.7350
Epoch 4/10
20/20 [==============================] - 42s 2s/step - loss: 0.6362 - accuracy: 0.7450 - val_loss: 0.5259 - val_accuracy: 0.8250
Epoch 5/10
20/20 [==============================] - 45s 2s/step - loss: 0.5211 - accuracy: 0.8175 - val_loss: 0.5245 - val_accuracy: 0.7900
Epoch 6/10
20/20 [==============================] - 46s 2s/step - loss: 0.6305 - accuracy: 0.8376 - val_loss: 0.6019 - val_accuracy: 0.7750
Epoch 7/10
20/20 [==============================] - 45s 2s/step - loss: 0.5212 - acc

In [7]:
from keras.preprocessing import image
import numpy as np
val_path = '/Users/utkarsh.verma/Downloads/archive/seg_test/street/24040.jpg'
img = image.load_img(val_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
block4_pool_features = model.predict(x)